# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
spaceship.shape

(8693, 14)

In [4]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [5]:
spaceship.isnull()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8689,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8690,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8691,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [6]:
spaceship.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:
spaceship_cleaned = spaceship.dropna()

In [8]:
spaceship_cleaned.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

In [9]:
spaceship_cleaned.shape

(6606, 14)

In [10]:
spaceship_cleaned['Cabin'].unique()

array(['B/0/P', 'F/0/S', 'A/0/S', ..., 'G/1499/S', 'G/1500/S', 'E/608/S'],
      dtype=object)

In [11]:
# Extraer solo la letra al inicio de cada valor en la columna "Cabin"
spaceship_cleaned['Cabin_section'] = spaceship_cleaned['Cabin'].str.extract(r'^([A-Z])')


print(spaceship_cleaned['Cabin_section'])


0       B
1       F
2       A
3       A
4       F
       ..
8688    A
8689    G
8690    G
8691    E
8692    E
Name: Cabin_section, Length: 6606, dtype: object


/tmp/ipykernel_26811/480163185.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship_cleaned['Cabin_section'] = spaceship_cleaned['Cabin'].str.extract(r'^([A-Z])')


In [12]:
spaceship_cleaned['Cabin_section'].value_counts()

Cabin_section
F    2152
G    1973
E     683
B     628
C     587
D     374
A     207
T       2
Name: count, dtype: int64

In [13]:
spaceship_cleaned = spaceship_cleaned.drop(['PassengerId', 'Name'], axis=1)

In [14]:
spaceship_cleaned.shape

(6606, 13)

In [15]:
print(spaceship_cleaned.columns)

Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Transported', 'Cabin_section'],
      dtype='object')


In [16]:
spaceship_cleaned.dtypes

HomePlanet        object
CryoSleep         object
Cabin             object
Destination       object
Age              float64
VIP               object
RoomService      float64
FoodCourt        float64
ShoppingMall     float64
Spa              float64
VRDeck           float64
Transported         bool
Cabin_section     object
dtype: object

In [17]:
# Doing dummies (like one hot encoding) for non-numerical columns
spaceship_cleaned = pd.get_dummies(spaceship_cleaned, columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Cabin_section'], drop_first=True)

In [18]:
spaceship_cleaned.head(5)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,...,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True,Cabin_section_B,Cabin_section_C,Cabin_section_D,Cabin_section_E,Cabin_section_F,Cabin_section_G,Cabin_section_T
0,39.0,0.0,0.0,0.0,0.0,0.0,False,True,False,False,...,False,True,False,True,False,False,False,False,False,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,False,False,False,...,False,True,False,False,False,False,False,True,False,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,True,False,False,...,False,True,True,False,False,False,False,False,False,False
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,False,False,False,...,False,True,False,False,False,False,False,True,False,False


In [19]:
target = spaceship_cleaned['Transported']
features = spaceship_cleaned.drop(columns = ['Transported'])

In [20]:
print(target)
print(features)

0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 6606, dtype: bool
       Age  RoomService  FoodCourt  ShoppingMall     Spa  VRDeck  \
0     39.0          0.0        0.0           0.0     0.0     0.0   
1     24.0        109.0        9.0          25.0   549.0    44.0   
2     58.0         43.0     3576.0           0.0  6715.0    49.0   
3     33.0          0.0     1283.0         371.0  3329.0   193.0   
4     16.0        303.0       70.0         151.0   565.0     2.0   
...    ...          ...        ...           ...     ...     ...   
8688  41.0          0.0     6819.0           0.0  1643.0    74.0   
8689  18.0          0.0        0.0           0.0     0.0     0.0   
8690  26.0          0.0        0.0        1872.0     1.0     0.0   
8691  32.0          0.0     1049.0           0.0   353.0  3235.0   
8692  44.0        126.0     4688.0           0.0     

**Perform Train Test Split**

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X = features
y = target

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [24]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (5284, 5323)
X_test shape: (1322, 5323)
y_train shape: (5284,)
y_test shape: (1322,)


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [25]:
# Scaling

from sklearn.preprocessing import MinMaxScaler

In [26]:
# Para los datos de entrada X
normalizer_X = MinMaxScaler()
features_scaled = normalizer_X.fit_transform(X)


normalizer_y = MinMaxScaler()

# Si y es un vector unidimensional, conviértelo a 2D
if isinstance(y, pd.Series):
    y = y.values.reshape(-1, 1)
target_scaled = normalizer_y.fit_transform(y)

# Convertir los datos normalizados a DataFrame para mantener etiquetas originales
X_scaled_df = pd.DataFrame(features_scaled, columns=X.columns)
y_scaled_df = pd.DataFrame(target_scaled, columns=['target'])

In [27]:


# Suponiendo que ya has escalado X_train de esta manera:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Deberías escalar X_test así también:
X_test_scaled = scaler.transform(X_test)  # Nota que solo utilizas .transform en X_test, no .fit_transform

In [28]:
print(X_scaled_df)
print(y_scaled_df)

           Age  RoomService  FoodCourt  ShoppingMall       Spa    VRDeck  \
0     0.493671     0.000000   0.000000      0.000000  0.000000  0.000000   
1     0.303797     0.010988   0.000302      0.002040  0.024500  0.002164   
2     0.734177     0.004335   0.119948      0.000000  0.299670  0.002410   
3     0.417722     0.000000   0.043035      0.030278  0.148563  0.009491   
4     0.202532     0.030544   0.002348      0.012324  0.025214  0.000098   
...        ...          ...        ...           ...       ...       ...   
6601  0.518987     0.000000   0.228726      0.000000  0.073322  0.003639   
6602  0.227848     0.000000   0.000000      0.000000  0.000000  0.000000   
6603  0.329114     0.000000   0.000000      0.152779  0.000045  0.000000   
6604  0.405063     0.000000   0.035186      0.000000  0.015753  0.159077   
6605  0.556962     0.012702   0.157247      0.000000  0.000000  0.000590   

      HomePlanet_Europa  HomePlanet_Mars  CryoSleep_True  Cabin_A/0/S  ...  \
0        

In [29]:
#your code here

In [30]:
from sklearn.model_selection import train_test_split

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y_scaled_df, test_size=0.2, random_state=42
)


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [31]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

In [32]:
#your code here

bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [33]:
bagging_reg.fit(X_scaled_df, y_scaled_df)



/home/silvia/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_bagging.py:505: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

In [34]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [35]:
# no funciona

# pred = bagging_reg.predict(X_test)

#print("MAE", mean_absolute_error(pred, y_test))
#print("RMSE", mean_squared_error(pred, y_test, squared=False))
#print("R2 score", bagging_reg.score(X_scaled_df, y_test))

In [36]:
pred = bagging_reg.predict(X_test_scaled)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
# Aquí también deberías usar X_test_scaled
print("R2 score", bagging_reg.score(X_test_scaled, y_test))

/home/silvia/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but BaggingRegressor was fitted with feature names
  warnings.warn(


MAE 0.21130354573944402
RMSE 0.3246715081868913


/home/silvia/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/silvia/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but BaggingRegressor was fitted with feature names
  warnings.warn(


R2 score 0.5782918754620465


In [37]:

from sklearn.metrics import r2_score, mean_squared_error
r2 = r2_score(y_test, pred)
mse = mean_squared_error(y_test, pred)
print(f'R2: {r2}, MSE: {mse}')

R2: 0.5782918754620465, MSE: 0.10541158822835062


- Random Forests

In [38]:
#your code here

- Gradient Boosting

In [39]:
#your code here

- Adaptive Boosting

In [40]:
#your code here

Which model is the best and why?

In [41]:
#comment here